# Networks: structure, evolution & processes
**Internet Analytics - Lab 2**

---

**Group:** W

**Names:**

* Olivier Cloux
* Thibault Urien
* Saskia Reiss

---

#### Instructions

*This is a template for part 3 of the lab. Clearly write your answers, comments and interpretations in Markodown cells. Don't forget that you can add $\LaTeX$ equations in these cells. Feel free to add or remove any cell.*

*Please properly comment your code. Code readability will be considered for grading. To avoid long cells of codes in the notebook, you can also embed long python functions and classes in a separate module. Don’t forget to hand in your module if that is the case. In multiple exercises, you are required to come up with your own method to solve various problems. Be creative and clearly motivate and explain your methods. Creativity and clarity will be considered for grading.*

---

## 2.3 Epdemics


#### Exercise 2.9: Simulate an epidemic outbreak

In [ ]:
# Imports
import epidemics_helper
import requests
import networkx as nx
from networkx.readwrite import json_graph
import json
import matplotlib.pyplot as plt

In [ ]:
with open('../data/nyc_augmented_network.json') as data_file:
	data = json.load(data_file)
	g = json_graph.node_link_graph(data)
	sir = epidemics_helper.SimulationSIR(g, beta=10.0, gamma=0.1)

---

### 2.3.1 Stop the apocalypse!

#### Exercise 2.10: Strategy 1

#### Exercise 2.11: Strategy 2